# Concurrency

Async kernel enables concurrent execution through the `Caller` class. 
It provides anyio compatible methods and classmethods for code execution 
in threads and asynchronous event loops. 


## Concurrency mode

Every message request received by the kernel is run with a message handler using one of the run modes [listed below](#run-mode).
The run mode is decided by the kernel method `get_run_mode`.

## Run mode

- blocking: Run the handler directly blocking the message loop
- queue: Run in a queue
- task: Run in a task
- thread: Run in a Caller thread


## `Kernel.get_run_mode`

The kernel decides the run mode dynamically with the method [get_run_mode][async_kernel.Kernel.get_run_mode].

In [ ]:
from async_kernel import utils
from async_kernel.typing import KernelConcurrencyMode, MsgType, RunMode

kernel = utils.get_kernel()

kernel.get_run_mode(MsgType.comm_msg)

If the `kernel.concurrency_mode` is changed, the run mode for a particluar `msg_type` might change.

In [ ]:
kernel.concurrency_mode = KernelConcurrencyMode.blocking
print(kernel.concurrency_mode, kernel.get_run_mode(MsgType.comm_msg))

kernel.concurrency_mode = KernelConcurrencyMode.default
print(kernel.concurrency_mode, kernel.get_run_mode(MsgType.comm_msg))

Below is a list of the run modes for the currently available concurrency modes.

!!! note

    `blocking` mode is roughly equivalent to how IpyKernel < 7.0 operates. 

In [ ]:
data = kernel.all_concurrency_run_modes()
try:
    import pandas as pd
except ImportError:
    print(data)
else:
    data = pd.DataFrame(data)
    data["RunMode"] = data.RunMode.str.replace("##", "")
    data = data.pivot(index="MsgType", columns=["KernelConcurrencyMode", "SocketID"], values="RunMode")  # noqa: PD010
    display(data)

## Execute request run mode

There are a few options to modify how code cells are run.

- Metadata
- [Directly in code]
- tag
- Message header (in custom messages)
!!! Warning

    Only Jupyter lab is known to allow concurrent execution of cells.

### Code for example

- **This example requires ipywidgets**
- **Ensure you are running an async kernel**

Lets define a function that we'll reuse for the remainder of the notebook.

In [ ]:
async def demo():
    import threading

    import anyio
    from ipywidgets import Button

    from async_kernel import Caller, utils

    print("Run mode:", utils.get_job()["run_mode"])

    print(f"Thread name: '{threading.current_thread().name}'")
    button = Button(description="Finish")
    event = anyio.Event()
    caller = Caller()  # Use caller so the `##thread` example works.
    # This is because widget messages are received by the shell in the main thread. The event is being waited in this thread.
    button.on_click(lambda _: caller.call_no_context(event.set))
    display(button)
    await event.wait()
    button.close()
    print(f"Finished ... thread name: '{threading.current_thread().name}'")
    return "Finished"

Lets run it normally (queue)

In [ ]:
await demo()

In [ ]:
##queue
# Tip: try running this cell while the previous cell is still busy.
await demo()

### Run mode: task
``` python
##task
...
```

The `task` mode instructs the kernel to execute the code in a task separate to the queue, Both `task` and `thread` execute modes can be started when the kernel is *busy executing*. There is no imposed limitation on the number of tasks (or threads) that can be run concurrently.

See also the [Caller](caller.ipynb#caller) example on how to call directly.

In [ ]:
RunMode.task  # noqa: B018  # Using the literal `RunMode` values directly is also possible. Though it may show up as a [Flake8 B018 issue](https://docs.astral.sh/ruff/rules/useless-expression/)
await demo()

### Run mode: thread
``` python
##thread
...
```


In [ ]:
# This time we'll use the tag to run the cell in a Thread
await demo()

In [ ]:
##thread
%callers # magic provided by async kernel